# Run MPNN on the complexes

### Imports

In [1]:
%load_ext lab_black
# Python standard library
from glob import glob
import os
import socket
import sys

# 3rd party library imports
import dask
import matplotlib.pyplot as plt
import pandas as pd
import pyrosetta
import numpy as np
import scipy
import seaborn as sns
from tqdm.auto import tqdm  # jupyter compatible progress bar

tqdm.pandas()  # link tqdm to pandas
# Notebook magic
# save plots in the notebook
%matplotlib inline
# reloads modules automatically before executing cells
%load_ext autoreload
%autoreload 2
print(f"running in directory: {os.getcwd()}")  # where are we?
print(f"running on node: {socket.gethostname()}")  # what node are we on?

running in directory: /mnt/home/pleung/projects/grad_reqs
running on node: dig82


### Insert the root of the crispy_shifty repo

In [2]:
sys.path.insert(0, "/projects/crispy_shifty")

### Run MPNN on the complexes

In [3]:
from crispy_shifty.utils.io import gen_array_tasks

simulation_name = "01_mpnn_around_ligand"
design_list_file = (
    "/home/pleung/projects/grad_reqs/00_pack_around_ligand/packed_binders.list"
)
output_path = os.path.join(os.getcwd(), f"{simulation_name}")
options = " ".join(
    [
        "out:level 200",
        "extra_res_fa /home/pleung/projects/grad_reqs/00_pack_around_ligand/afo.params",
    ]
)
extra_kwargs = {
    "num_sequences": "96",
    "ligand_params": "/home/pleung/projects/grad_reqs/00_pack_around_ligand/afo.params",
}

gen_array_tasks(
    distribute_func="deployables.mpnn_around_ligand",
    func_root="/home/pleung/projects/grad_reqs",
    design_list_file=design_list_file,
    output_path=output_path,
    queue="gpu-remote",
    memory="24G",
    gres="--gres=gpu:1",
    nstruct=1,
    nstruct_per_task=1000,
    options=options,
    extra_kwargs=extra_kwargs,
    simulation_name=simulation_name,
)

/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


https://docs.anaconda.com/anaconda/install

Run the following command with your desired environment active:
sbatch -a 1-48 /mnt/home/pleung/projects/grad_reqs/01_mpnn_around_ligand/run.sh


### Collect scorefiles of the MPNN designs

In [3]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import collect_score_file

simulation_name = "01_mpnn_around_ligand"
output_path = os.path.join(os.getcwd(), f"{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.json")):
    collect_score_file(output_path, "scores")

https://docs.anaconda.com/anaconda/install



/projects/crispy_shifty/envs/crispy/lib/python3.8/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


### Load resulting concatenated scorefile

In [4]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import parse_scorefile_linear

output_path = os.path.join(os.getcwd(), f"{simulation_name}")

if not os.path.exists(os.path.join(output_path, "scores.csv")):
    scores_df = parse_scorefile_linear(os.path.join(output_path, "scores.json"))

### Dump scores_df as a CSV and then reload, for performance reasons

In [5]:
if not os.path.exists(os.path.join(output_path, "scores.csv")):
    scores_df.to_csv(os.path.join(output_path, "scores.csv"))

scores_df = pd.read_csv(os.path.join(output_path, "scores.csv"), index_col="Unnamed: 0")

### Data exploration

In [6]:
print(len(scores_df))
print(list(scores_df.columns))

94084
['dslf_fa13', 'fa_atr', 'fa_dun', 'fa_dun_dev', 'fa_dun_rot', 'fa_dun_semi', 'fa_elec', 'fa_intra_atr_xover4', 'fa_intra_elec', 'fa_intra_rep', 'fa_intra_rep_xover4', 'fa_intra_sol_xover4', 'fa_rep', 'fa_sol', 'hbond_bb_sc', 'hbond_lr_bb', 'hbond_sc', 'hbond_sr_bb', 'hxl_tors', 'lk_ball', 'lk_ball_bridge', 'lk_ball_bridge_uncpl', 'lk_ball_iso', 'lk_ball_wtd', 'omega', 'p_aa_pp', 'path_in', 'pro_close', 'rama_prepro', 'ref', 'total_score', 'type', 'yhh_planarity', 'mpnn_seq_0000', 'mpnn_seq_0001', 'mpnn_seq_0002', 'mpnn_seq_0003', 'mpnn_seq_0004', 'mpnn_seq_0005', 'mpnn_seq_0006', 'mpnn_seq_0007', 'mpnn_seq_0008', 'mpnn_seq_0009', 'mpnn_seq_0010', 'mpnn_seq_0011', 'mpnn_seq_0012', 'mpnn_seq_0013', 'mpnn_seq_0014', 'mpnn_seq_0015', 'mpnn_seq_0016', 'mpnn_seq_0017', 'mpnn_seq_0018', 'mpnn_seq_0019', 'mpnn_seq_0020', 'mpnn_seq_0021', 'mpnn_seq_0022', 'mpnn_seq_0023', 'mpnn_seq_0024', 'mpnn_seq_0025', 'mpnn_seq_0026', 'mpnn_seq_0027', 'mpnn_seq_0028', 'mpnn_seq_0029', 'mpnn_seq_0030',

### Save individual fastas

In [15]:
sys.path.insert(0, "/projects/crispy_shifty")
from crispy_shifty.utils.io import df_to_fastas

output_path = os.path.join(os.getcwd(), f"{simulation_name}")

scores_df = df_to_fastas(scores_df, prefix="mpnn_seq")

  0%|          | 0/94084 [00:00<?, ?it/s]

### Save a list of outputs

In [16]:
simulation_name = "01_mpnn_around_ligand"
output_path = os.path.join(os.getcwd(), f"{simulation_name}")

with open(os.path.join(output_path, "mpnn_designed.list"), "w") as f:
    for path in tqdm(scores_df.index):
        print(path, file=f)

  0%|          | 0/94084 [00:00<?, ?it/s]

### Concat the pdb.bz2 and fasta paths into a single list, for Superfold reasons

In [17]:
simulation_name = "01_mpnn_around_ligand"
output_path = os.path.join(os.getcwd(), f"{simulation_name}")

with open(os.path.join(output_path, "mpnn_designed.pair"), "w") as f:
    for path in tqdm(scores_df.index):
        line = path + "____" + path.replace("decoys", "fastas").replace("pdb.bz2", "fa")
        print(line, file=f)

  0%|          | 0/94084 [00:00<?, ?it/s]

### Get only BenTF2s
Otherwise it doesn't count

In [12]:
filtered = scores_df.query("path_in.str.contains('beNTF2')", engine="python")
len(filtered)

61794

### Save a list of BenTF2s

In [18]:
simulation_name = "01_mpnn_around_ligand"
output_path = os.path.join(os.getcwd(), f"{simulation_name}")

with open(os.path.join(output_path, "filtered_designed.list"), "w") as f:
    for path in tqdm(filtered.index):
        print(path, file=f)

  0%|          | 0/61794 [00:00<?, ?it/s]

### Concat the pdb.bz2 and fasta paths into a single list, for Superfold reasons

In [19]:
simulation_name = "01_mpnn_around_ligand"
output_path = os.path.join(os.getcwd(), f"{simulation_name}")

with open(os.path.join(output_path, "filtered_designed.pair"), "w") as f:
    for path in tqdm(filtered.index):
        line = path + "____" + path.replace("decoys", "fastas").replace("pdb.bz2", "fa")
        print(line, file=f)

  0%|          | 0/61794 [00:00<?, ?it/s]

### Prototyping blocks